In [1]:
import pandas as pd
jurnal = pd.read_csv(r"C:\Users\Fauzan Anshori\Downloads\Jurnal ML\datajurnal2.csv",sep=";")

In [2]:
import nltk
import nltk.tokenize


# Tokenizing
nltk.download('punkt')
# Stopwords removal
nltk.download('stopwords')
# Lemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

print('Completed')

Completed


[nltk_data] Downloading package punkt to C:\Users\Fauzan
[nltk_data]     Anshori\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Fauzan
[nltk_data]     Anshori\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Fauzan
[nltk_data]     Anshori\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Fauzan
[nltk_data]     Anshori\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
import re

def clean_title(title):
  re.sub("[^a-zA-Z0-9 ]", "", title)
  return title

def clean_abstract(abstract):
  re.sub("[a-zA-Z ]", "", abstract)
  return abstract

jurnal["clean_title"] = jurnal["title"].apply(clean_title)
jurnal["clean_abstract"] = jurnal["abstract"].apply(clean_abstract)

In [4]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(jurnal["clean_abstract"])

cosine_sim_matrix = cosine_similarity(tfidf_matrix)

with open('recommendation_model.pkl', 'wb') as f:
    pickle.dump({
        'vectorizer': vectorizer,
        'cosine_similarity': cosine_sim_matrix,
        'documents': jurnal["clean_abstract"],
        'titles': jurnal["clean_title"]
    }, f)

print("Model saved as 'recommendation_model.pkl'")

Model saved as 'recommendation_model.pkl'


In [6]:
import pickle
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Load the saved TF-IDF vectorizer and documents from the pickle file
with open('recommendation_model.pkl', 'rb') as f:
    model_data = pickle.load(f)

# Extract the vectorizer, cosine similarity matrix, and documents
loaded_vectorizer = model_data['vectorizer']
loaded_cosine_sim = model_data['cosine_similarity']
loaded_documents = model_data['documents']
loaded_titles = model_data['titles']

# Step 2: Function to recommend similar documents based on cosine similarity
def get_recommendations(cosine_sim_matrix, titles, top_n=5):
    # Sort documents based on similarity scores in descending order
    sim_scores = list(enumerate(cosine_sim_matrix[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[:top_n]  # Exclude the first one (itself)
    # first_indices = [item[0] for item in sim_scores]
    # Return the most similar documentsd
    # print(first_indices)
    return [titles[i[0]] for i in sim_scores]

# Step 3: Take user input
user_input = input("Enter the journal text you are interested in: ")

# Step 4: Transform the user input into a TF-IDF vector
user_tfidf = loaded_vectorizer.transform([user_input])

# Step 5: Compute the cosine similarity between user input and stored documents
user_cosine_sim = cosine_similarity(user_tfidf, loaded_vectorizer.transform(loaded_documents))

# Step 6: Get recommendations based on the user's input
recommendations = get_recommendations(user_cosine_sim, loaded_titles)

# Step 7: Display the recommendations
print("Recommended Journals/Articles:")
for rec in recommendations:
    print(rec)

Recommended Journals/Articles:
Mapping cancer origins
An overview of internet of things
Cancer and cure: A critical analysis
Existence and Asymptotic Behavior of Radially Symmetric Solutions to a Semilinear Hyperbolic System in Odd Space Dimensions∗∗∗
Pathophysiology of lung cancer
